In [1]:
import pandas as pd 
import numpy as np 
import os
import cv2
import matplotlib.pyplot as plt
import warnings
import tensorflow 
import scipy

from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout, BatchNormalization, Input, AveragePooling2D,Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image, image_dataset_from_directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential,Model
from tensorflow import keras
import tensorflow as tf

In [3]:
train_path = './dataset/train'

In [10]:
batch_size = 8
img_height = 256
img_width = 256

In [11]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  train_path,
  validation_split=0.2,
  subset="training",
  seed=123,
  label_mode='categorical',
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 5121 files belonging to 4 classes.
Using 4097 files for training.


In [12]:
val_ds = tf.keras.utils.image_dataset_from_directory(
  train_path,
  validation_split=0.2,
  subset="validation",
  seed=123,
  label_mode='categorical',
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 5121 files belonging to 4 classes.
Using 1024 files for validation.


In [13]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [14]:
rn = ResNet50(input_shape=(256,256,3), weights='imagenet', include_top=False)
for layer in rn.layers:
    layer.trainable = False
x = Flatten()(rn.output)

prediction = Dense(4, activation='softmax')(x)

modelrn = Model(inputs=rn.input, outputs=prediction)

modelrn.compile(optimizer='adam',
loss=tensorflow.losses.CategoricalCrossentropy(),
metrics=[keras.metrics.AUC(name='auc')])

callback = keras.callbacks.EarlyStopping(monitor='val_loss',patience=8,restore_best_weights=True)

In [15]:
modelrn.fit(train_ds, epochs=15, validation_data=val_ds,callbacks=callback)

Epoch 1/15
513/513 [==============================] - 144s 216ms/step - loss: 9.2019 - auc: 0.7647 - val_loss: 5.0389 - val_auc: 0.8392
Epoch 2/15
513/513 [==============================] - 107s 208ms/step - loss: 3.3542 - auc: 0.9008 - val_loss: 3.1306 - val_auc: 0.8955
Epoch 3/15
513/513 [==============================] - 103s 201ms/step - loss: 1.9856 - auc: 0.9359 - val_loss: 4.9754 - val_auc: 0.8891
Epoch 4/15
513/513 [==============================] - 103s 200ms/step - loss: 1.6901 - auc: 0.9477 - val_loss: 2.0942 - val_auc: 0.9290
Epoch 5/15
513/513 [==============================] - 102s 200ms/step - loss: 1.7769 - auc: 0.9529 - val_loss: 5.7951 - val_auc: 0.8612
Epoch 6/15
513/513 [==============================] - 164s 320ms/step - loss: 1.2730 - auc: 0.9664 - val_loss: 4.5368 - val_auc: 0.9106
Epoch 7/15
513/513 [==============================] - 102s 199ms/step - loss: 1.3135 - auc: 0.9666 - val_loss: 5.7759 - val_auc: 0.8866
Epoch 8/15
513/513 [============================

In [16]:
loss, accuracy = modelrn.evaluate(val_ds)
print("Accuracy: ", accuracy)

128/128 [==============================] - 21s 161ms/step - loss: 2.0942 - auc: 0.9290
Accuracy:  0.928955614566803
